In [1]:
from __future__ import absolute_import, division, print_function

import argparse
import os
import random
from datetime import datetime

import cv2
import keras
import numpy as np
import pandas as pd
from keras import backend as K
from keras.applications import *
from keras.applications.inception_v3 import preprocess_input
from keras.callbacks import *
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import *
from keras.utils.vis_utils import model_to_dot
from tqdm import tqdm

Using TensorFlow backend.


In [24]:
def load_data():
    df = pd.read_csv('../dog_breed_datasets/labels.csv')
    df.head()

    n = len(df)
    breed = set(df['breed'])
    n_class = len(breed)
    class_to_num = dict(zip(breed, range(n_class)))
    num_to_class = dict(zip(range(n_class), breed))

    width = 299
    X = np.zeros((n, width, width, 3), dtype=np.float16)
    y = np.zeros((n, n_class), dtype=np.uint8)
    # Loading Datasets
    print('\n\n Loading Datasets. \n')
    for i in tqdm(range(n)):
        X[i] = (cv2.resize(
            cv2.imread('../dog_breed_datasets/train/%s.jpg' % df['id'][i]),
            (width, width))/255. - 0.5)*2
        y[i][class_to_num[df['breed'][i]]] = 1

    dvi = int(X.shape[0] * 0.9)
    x_train = X[:dvi, :, :, :]
    y_train = y[:dvi, :]
    x_val = X[dvi:, :, :, :]
    y_val = y[dvi:, :]

  0%|          | 30/10222 [00:00<00:34, 292.31it/s]



 Loading Datasets. 






















































  1%|          | 80/10222 [00:00<00:38, 262.68it/s]

  1%|▏         | 134/10222 [00:00<00:38, 264.79it/s]

  2%|▏         | 185/10222 [00:00<00:38, 260.31it/s]

...

 98%|█████████▊| 10050/10222 [00:38<00:00, 258.31it/s]

 99%|█████████▉| 10105/10222 [00:39<00:00, 258.35it/s]

 99%|█████████▉| 10159/10222 [00:39<00:00, 258.28it/s]

100%|█████████▉| 10211/10222 [00:39<00:00, 258.21it/s]

100%|██████████| 10222/10222 [00:39<00:00, 258.20it/s]

In [29]:
x_train = np.load('../dog_breed_datasets/x_train.npy')
y_train = np.load('../dog_breed_datasets/y_train.npy')
x_val = np.load('../dog_breed_datasets/x_val.npy')
y_val = np.load('../dog_breed_datasets/y_val.npy')

In [30]:
x_train.max()

1.0

In [3]:
def run(model_name, lr, optimizer, epoch, patience, batch_size, test=None):
    width = x_train.shape[1]
    n_class = y_train.shape[1]
    # Compute the bottleneck feature
    def get_features(MODEL, data=x_train):
        cnn_model = MODEL(
            include_top=False,
            input_shape=(width, width, 3),
            weights='imagenet')
        inputs = Input((width, width, 3))
        x = inputs
        x = Lambda(preprocess_input, name='preprocessing')(x)
        x = cnn_model(x)
        x = GlobalAveragePooling2D()(x)
        cnn_model = Model(inputs, x)

        features = cnn_model.predict(data, batch_size=32, verbose=1)
        return features

    def fine_tune(MODEL,
                  model_name,
                  optimizer,
                  lr,
                  epoch,
                  patience,
                  batch_size,
                  X=x_train,
                  test=None):
        # Fine-tune the model
        print("\n\n Fine tune " + model_name + " : \n")

        from random_eraser import get_random_eraser
        datagen = ImageDataGenerator(
            preprocessing_function=get_random_eraser(pixel_level=True),
            horizontal_flip=True,
            shear_range=0.1,
            zoom_range=0.1,
            rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1)

        val_datagen = ImageDataGenerator()

        inputs = Input((width, width, 3))
        x = inputs
        cnn_model = MODEL(
            include_top=False,
            input_shape=(width, width, 3),
            weights='imagenet')
        x = cnn_model(x)
        x = GlobalAveragePooling2D()(x)
        x = Dropout(0.5)(x)
        x = Dense(n_class, activation='softmax', name='predictions')(x)
        model = Model(inputs=inputs, outputs=x)

        # Loading weights
        try:
            model.load_weights(model_name + '.h5')
            print('Load ' + model_name + '.h5 successfully.')
        except:
            try:
                model.load_weights('fc_' + model_name + '.h5', by_name=True)
                print('Fail to load ' + model_name + '.h5, load fc_' +
                      model_name + '.h5 instead.')
            except:
                print(
                    'Start computing ' + model_name + ' bottleneck feature: ')
                features = get_features(MODEL, X)

                # Training models
                inputs = Input(features.shape[1:])
                x = inputs
                x = Dropout(0.5)(x)
                x = Dense(n_class, activation='softmax', name='predictions')(x)
                model_fc = Model(inputs, x)
                model_fc.compile(
                    optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
                h = model_fc.fit(
                    features,
                    y_train,
                    batch_size=128,
                    epochs=5,
                    validation_split=0.1)

                model_fc.save('fc_' + model_name + '.h5', 'w')

        print("\n " + "Optimizer=" + optimizer + " lr=" + str(lr) + " \n")
        if optimizer == "Adam":
            model.compile(
                optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
        elif optimizer == "SGD":
            model.compile(
                loss='categorical_crossentropy',
                optimizer=SGD(lr=lr, momentum=0.9, nesterov=True),
                metrics=['accuracy'])
        if not test:
            class LossHistory(keras.callbacks.Callback):
                def on_train_begin(self, logs={}):
                    self.losses = []
                def on_epoch_end(self, batch, logs={}):
                    self.losses.append((logs.get('loss'), logs.get("val_loss")))

            history = LossHistory()

            early_stopping = EarlyStopping(
                monitor='val_loss', patience=patience, verbose=1, mode='auto')
            checkpointer = ModelCheckpoint(
                filepath=model_name + '.h5', verbose=0, save_best_only=True)
            reduce_lr = ReduceLROnPlateau(factor=0.2, patience=3, verbose=1)
            model.fit_generator(
                datagen.flow(x_train, y_train, batch_size=batch_size),
                steps_per_epoch=len(x_train) / batch_size,
                validation_data=val_datagen.flow(
                    x_val, y_val, batch_size=batch_size),
                validation_steps=len(x_val) / batch_size,
                epochs=epoch,
                callbacks=[history, early_stopping, checkpointer, reduce_lr])
            with open(model_name + ".csv", 'a') as f_handle:
                np.savetxt(f_handle, history.losses)
        else:
            print('Evalute on test set')
            val_datagen.fit(x_test)
            score = model.evaluate_generator(
                val_datagen.flow(x_test, y_test, batch_size=batch_size),
                len(x_test) / batch_size)
            print(score)
            return score

    list_model = {
        "Xception": Xception,
        "InceptionV3": InceptionV3,
        "InceptionResNetV2": InceptionResNetV2
    }
    fine_tune(list_model[model_name], model_name, optimizer, lr, epoch,
              patience, batch_size, x_train, test)


In [5]:
run('Xception', 5e-4, 'SGD', 1e4, 7, 16)



 Fine tune Xception : 

Fail to load Xception.h5, load fc_Xception.h5 instead.

 Optimizer=SGD lr=0.0005 

Epoch 1/10000
183/574 [========>.....................] - ETA: 5:39 - loss: 1.0928 - acc: 0.6938

ResourceExhaustedError: OOM when allocating tensor with shape[16,10,10,2048]
	 [[Node: training_1/SGD/gradients/global_average_pooling2d_2/Mean_grad/Tile = Tile[T=DT_FLOAT, Tmultiples=DT_INT32, _class=["loc:@global_average_pooling2d_2/Mean"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_1/SGD/gradients/global_average_pooling2d_2/Mean_grad/Reshape, training_1/SGD/gradients/global_average_pooling2d_2/Mean_grad/floordiv)]]

Caused by op u'training_1/SGD/gradients/global_average_pooling2d_2/Mean_grad/Tile', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/u